# Importações

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.figure_factory as ff

from plotly.subplots import make_subplots

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.nonparametric.smoothers_lowess import lowess

# Leitura dos dados

In [2]:
folder_data = '../data'
clima = pd.read_csv(f'{folder_data}/clima.csv',parse_dates=['date'])
consumo = pd.read_csv(f'{folder_data}/consumo.csv',parse_dates=['date'])
clientes = pd.read_csv(f'{folder_data}/clientes.csv')

# Dados de Clientes

In [3]:
display(clientes.head(10))
print(clientes.info())

# Tipos de dados corretos
# Sem dados faltantes

,client_id,region
0,C0000,Norte
1,C0001,Norte
2,C0002,Sul
3,C0003,Norte
4,C0004,Oeste
5,C0005,Norte
6,C0006,Centro
7,C0007,Centro
8,C0008,Centro
9,C0009,Centro


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  100 non-null    object
 1   region     100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
None


In [4]:
print(clientes['region'].drop_duplicates())
print(len(clientes[clientes['region'] == 'Desconhecida']))

# Região 'Desconhecida' é um erro de digitação
# A quantidade de clientes com região 'Desconhecida' é pequena, a decisão será tomada após algumas análises

0            Norte
2              Sul
4            Oeste
6           Centro
10           Leste
15    Desconhecida
Name: region, dtype: object
5


In [5]:
# Verifica se cada cliente está associado a apenas uma região
clientes_regioes = clientes.groupby('client_id')['region'].nunique()
clientes_varias_regioes = clientes_regioes[clientes_regioes > 1]

print(f"Número de clientes com mais de uma região: {len(clientes_varias_regioes)}")
if not clientes_varias_regioes.empty:
    print("IDs dos clientes com mais de uma região:")
    print(clientes_varias_regioes.index.tolist())
else:
    print("Todos os clientes possuem apenas uma região associada.")

Número de clientes com mais de uma região: 0
Todos os clientes possuem apenas uma região associada.


In [6]:
clientes_por_regiao = (
    clientes['region']
    .value_counts()
    .reset_index()
    .rename(columns={'quantidade': 'region', 'count': 'quantidade'})
)

fig = px.bar(
    clientes_por_regiao,
    x='region',
    y='quantidade',
    title='Quantidade de clientes por região',
    labels={'region': 'Região', 'quantidade': 'Quantidade de Clientes'},
    text='quantidade'
)
fig.update_layout(template='simple_white')
fig.show()

In [7]:
dados = clientes[clientes['region'] == 'Desconhecida']

dados = (
    dados
    .merge(
        consumo,
        on='client_id',
        how='inner'
    )
    .groupby(['client_id','date'])
    .sum()
    .reset_index()
)

In [8]:
fig = make_subplots(rows=1, cols=1)

for client in dados['client_id'].unique():
    cliente_dados = dados[dados['client_id'] == client]

    fig.add_trace(
        go.Scatter(
            x=cliente_dados['date'],
            y=cliente_dados['consumption_kwh'],
            mode='lines',
            name=f'{client}',
            line=dict(dash='solid')
        ),
        row=1, col=1
    )


# Ajusta layout
fig.update_layout(
    height=700,
    title_text="Séries Temporais de Clientes com Região 'Desconhecida'",
    template='simple_white',
    xaxis_title="Data",
    yaxis_title="Consumo (kWh)"
)

fig.show()

In [9]:
# Os clientes com região 'Desconhecida' não apresentam padrão de consumo entre eles.
# A primeira alternativa a ser tomada é criar um modelo de classificação para prever a região de cada cliente.
# A segunda alternativa é remover esses clientes do dataset, pois a quantidade é pequena e não há padrão de consumo entre eles.

In [10]:
clientes = clientes[clientes['region'] != 'Desconhecida']

# Dados de Clima

In [11]:
display(clima.head(10))
print(clima.info())

# Tipos de dados corretos
# Dados faltantes em temperature
# Dados tem frequência diária

,region,date,temperature,humidity
0,Centro,2023-01-01,25.652941,57.188235
1,Centro,2023-01-02,23.794118,52.517647
2,Centro,2023-01-03,22.900000,61.358824
3,Centro,2023-01-04,24.158824,64.223529
4,Centro,2023-01-05,24.870588,59.058824
5,Centro,2023-01-06,26.194118,59.629412
6,Centro,2023-01-07,23.905882,61.752941
7,Centro,2023-01-08,25.900000,60.117647
8,Centro,2023-01-09,25.688235,57.964706
9,Centro,2023-01-10,23.947059,54.088235


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   region       900 non-null    object        
 1   date         900 non-null    datetime64[ns]
 2   temperature  855 non-null    float64       
 3   humidity     900 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 28.2+ KB
None


In [12]:
# Interpolação dos valores nulos de temperature
concat_df = []
for region in clima['region'].drop_duplicates():
    df_region = clima[clima['region'] == region].copy()
    df_region = df_region.set_index('date')
    df_region['temperature'] = df_region['temperature'].interpolate(method='time')
    df_region['region'] = region
    df_region = df_region.reset_index()
    df_region['temperature'] = df_region['temperature'].bfill()
    concat_df.append(df_region)

clima = pd.concat(concat_df)

# Interpolação preenche os valores ausentes com base nos dados vizinhos, mantendo o padrão e a fluidez da série temporal.
# Isso é crucial para modelos que consideram sequências de tempo.
# Preencher com média, zero ou mediana ignora a dinâmica da série.
# Interpolação respeita a tendência local

In [13]:
display(clima.describe())

# Dados diários sem dias futuros
# Dados de temperature e humidity sem valores exorbitantes

,date,temperature,humidity
count,900,900.000000,900.000000
mean,2023-03-31 11:59:59.999999744,25.105312,60.013529
min,2023-01-01 00:00:00,21.313333,52.517647
25%,2023-02-14 18:00:00,24.343056,58.456111
50%,2023-03-31 12:00:00,25.036397,60.005556
75%,2023-05-15 06:00:00,25.875000,61.565196
max,2023-06-29 00:00:00,28.746667,69.111765
std,NaN,1.163087,2.421545


In [14]:
clima[['temperature','humidity']].corr()

# não há correlação forte entre temperature e humidity

,temperature,humidity
temperature,1.000000,0.027146
humidity,0.027146,1.000000


In [15]:
clima = clima.sort_values(by=['region', 'date'])

regioes = clima['region'].unique()
fig = sp.make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    subplot_titles=("Série temporal de Temperatura por Região", "Série temporal de Umidade por Região")
)

# Gráfico de temperatura por região (mantendo as horas)
for reg in regioes:
    dados = clima[clima['region'] == reg]
    fig.add_trace(
        go.Scatter(
            x=dados['date'],  # Mantém as horas
            y=dados['temperature'],
            mode='lines',
            name=f'Temperatura - {reg}'
        ),
        row=1, col=1
    )

# Gráfico de umidade por região (mantendo as horas)
for reg in regioes:
    dados = clima[clima['region'] == reg]
    fig.add_trace(
        go.Scatter(
            x=dados['date'],  # Mantém as horas
            y=dados['humidity'],
            mode='lines',
            name=f'Umidade - {reg}',
            line=dict(dash='dot')
        ),
        row=2, col=1
    )

fig.update_layout(
    height=700,
    title_text="Séries temporais de Temperatura e Umidade por Região",
    template='simple_white'
)
fig.update_yaxes(title_text="Temperatura (°C)", row=1, col=1)
fig.update_yaxes(title_text="Umidade (%)", row=2, col=1)
fig.update_xaxes(title_text="Data", row=2, col=1)

fig.show()

In [16]:
# Temperatura: entre 22°C e 29°C, com variações diárias esperadas, mas sem tendência clara de aumento ou queda ao longo dos meses.
# Umidade: varia entre 52% e 70%, também com certa estabilidade, embora pareça haver redução leve na variabilidade após abril.

# Algumas quedas rápidas de temperatura e picos de umidade ocorrem esporadicamente, o que pode indicar:
# Mudanças de clima súbitas
# Frentes frias ou chuvas
# Instabilidades regionais

# Visivelmente, não há uma correlação direta forte entre temperatura e umidade diária.
# Porém, em alguns momentos, é possível observar que quando a temperatura cai, a umidade sobe, o que pode ser característico de eventos climáticos específicos.

In [17]:
regioes = clima['region'].unique()
resultados = {}

# Calcula a decomposição para cada região e armazena os resultados
for reg in regioes:
    serie = clima[clima['region'] == reg].set_index('date')['temperature']
    resultado = seasonal_decompose(serie, model='additive', period=28)
    resultados[reg] = resultado

# Gráfico 1: Todas as tendências
fig_trend = go.Figure()
for reg in regioes:
    fig_trend.add_trace(go.Scatter(
        x=resultados[reg].trend.index,
        y=resultados[reg].trend,
        mode='lines',
        name=f'Tendência - {reg}'
    ))
fig_trend.update_layout(
    title='Tendências da Temperatura por Região',
    xaxis_title='Data',
    yaxis_title='Tendência',
    template='simple_white'
)
fig_trend.show()

# Gráfico 2: Todas as sazonalidades
fig_seasonal = go.Figure()
for reg in regioes:
    fig_seasonal.add_trace(go.Scatter(
        x=resultados[reg].seasonal.index,
        y=resultados[reg].seasonal,
        mode='lines',
        name=f'Sazonalidade - {reg}'
    ))
fig_seasonal.update_layout(
    title='Sazonalidades da Temperatura por Região',
    xaxis_title='Data',
    yaxis_title='Sazonalidade',
    template='simple_white'
)
fig_seasonal.show()


In [18]:
# Está decomposição mostra a tendência, sazonalidade e resíduo da temperatura.
# Não há um tendência clara de aumento ou queda ao longo dos meses.
# Há sazonalidade mensal, com picos e vales que se repetem a cada 30 dias.

In [19]:
regioes = clima['region'].unique()
resultados = {}

# Calcula a decomposição para cada região e armazena os resultados
for reg in regioes:
    serie = clima[clima['region'] == reg].set_index('date')['humidity']
    resultado = seasonal_decompose(serie, model='additive', period=28)
    resultados[reg] = resultado

# Gráfico 1: Todas as tendências
fig_trend = go.Figure()
for reg in regioes:
    fig_trend.add_trace(go.Scatter(
        x=resultados[reg].trend.index,
        y=resultados[reg].trend,
        mode='lines',
        name=f'Tendência - {reg}'
    ))
fig_trend.update_layout(
    title='Tendências da Umidade por Região',
    xaxis_title='Data',
    yaxis_title='Tendência',
    template='simple_white'
)
fig_trend.show()

# Gráfico 2: Todas as sazonalidades
fig_seasonal = go.Figure()
for reg in regioes:
    fig_seasonal.add_trace(go.Scatter(
        x=resultados[reg].seasonal.index,
        y=resultados[reg].seasonal,
        mode='lines',
        name=f'Sazonalidade - {reg}'
    ))
fig_seasonal.update_layout(
    title='Sazonalidades da Umidade por Região',
    xaxis_title='Data',
    yaxis_title='Sazonalidade',
    template='simple_white'
)
fig_seasonal.show()


In [20]:
# Está decomposição mostra a tendência, sazonalidade e resíduo da umidade.
# Não há um tendência clara de aumento ou queda ao longo dos meses.
# Há sazonalidade mensal, com picos e vales que se repetem a cada 30 dias.

# Dados de Consumo

In [21]:
display(consumo.head(10))
print(consumo.info())

# Tipos de dados corretos
# Sem dados faltantes

,client_id,date,consumption_kwh
0,C0000,2023-01-01,18.64
1,C0000,2023-01-02,16.63
2,C0000,2023-01-03,18.11
3,C0000,2023-01-04,18.25
4,C0000,2023-01-05,19.81
5,C0000,2023-01-06,15.87
6,C0000,2023-01-07,20.30
7,C0000,2023-01-08,19.35
8,C0000,2023-01-09,18.30
9,C0000,2023-01-10,13.34


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   client_id        18000 non-null  object        
 1   date             18000 non-null  datetime64[ns]
 2   consumption_kwh  18000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 422.0+ KB
None


In [22]:
display(consumo.describe())

# Dados diários sem dias futuros
# Dados de Consumo sem valores exorbitantes

,date,consumption_kwh
count,18000,18000.000000
mean,2023-03-31 11:59:59.999999744,14.811876
min,2023-01-01 00:00:00,2.640000
25%,2023-02-14 18:00:00,12.070000
50%,2023-03-31 12:00:00,14.890000
75%,2023-05-15 06:00:00,17.630000
max,2023-06-29 00:00:00,27.920000
std,NaN,3.728855


In [23]:
dias_por_cliente = consumo.groupby('client_id')['date'].nunique()
todos_tem_180_dias = dias_por_cliente.eq(180).all()
print(f"Todos os client_id têm 180 dias? {todos_tem_180_dias}")

Todos os client_id têm 180 dias? True


In [24]:
duplicados = consumo.duplicated(subset=['client_id', 'date'], keep=False)
tem_duplicados = duplicados.any()
print(f"Há valores de dias duplicados para algum client_id? {tem_duplicados}")

Há valores de dias duplicados para algum client_id? False


In [25]:
fig = px.histogram(
    consumo,
    x='consumption_kwh', 
    nbins=50, 
    title='Distribuição do consumo de energia (kWh)',
    labels={'consumption_kwh': 'Consumo (kWh)'},
    marginal='box',
    opacity=0.75
)

fig.update_layout(
    xaxis_title='Consumo (kWh)',
    yaxis_title='Frequência',
    bargap=0.05,
    template='simple_white'
)

fig.show()

In [26]:
# O gráfico mostra que a maioria dos clientes consome entre 8 e 20 kWh, centrada entre 12.07 e 17.63
# Não há valores exorbitantes, ou seja, sem a presença de outliers extremos. Há poucos valores abaixo de 3.81 e acima de 25.73
# Isso sugere que a maioria dos clientes tem um padrão de consumo diário "típico"

In [27]:
consumo_diario = (
    consumo
    .groupby('date')['consumption_kwh']
    .sum()
    .reset_index()
)

fig = px.line(
    consumo_diario,
    x='date',
    y='consumption_kwh',
    title='Consumo total diário de energia',
    labels={'date': 'Data', 'consumption_kwh': 'Consumo (kWh)'}
)

# Seleciona finais de semana
finais_semana = consumo_diario[consumo_diario['date'].dt.weekday >= 5]

fig.add_scatter(
    x=finais_semana['date'],
    y=finais_semana['consumption_kwh'],
    mode='markers',
    marker=dict(color='red', size=8),
    name='Final de Semana'
)

fig.update_layout(
    xaxis_title='Data',
    yaxis_title='Consumo (kWh)',
    template='simple_white'
)

fig.show()

In [28]:
# Este gráfico mostra o consumo total diário de energia ao longo do tempo, com destaque para os finais de semana.
# É possível observar que o final de semana não é um fator impactante no consumo de energia, pois não há um aumento ou decréscimo significativo nos valores de consumo nesses dias.
# Não há uma tendência de alta ou queda consistente ao longo dos meses.
# Alguns dias apresentam picos isolados de consumo, que podem indicar eventos climáticos extremos

In [29]:
# Adiciona coluna indicando se é final de semana
serie_df = consumo_diario.reset_index()
serie_df['dia_semana'] = serie_df['date'].dt.dayofweek
serie_df['tipo_dia'] = np.where(serie_df['dia_semana'] >= 5, 'Final de Semana', 'Dia de Semana')

fig = px.box(
    serie_df,
    x='tipo_dia',
    y='consumption_kwh',
    color='tipo_dia',
    title='Boxplot do Consumo de Energia: Dias de Semana vs Finais de Semana',
    labels={'tipo_dia': 'Tipo de Dia', 'consumption_kwh': 'Consumo (kWh)'},
    template='simple_white'
)
fig.update_layout(showlegend=False)
fig.show()

In [30]:
# Este gráfico reafirma que finais de semana não apresentam um padrão de consumo distinto em relação aos dias de semana.

In [31]:
# Série temporal: índice deve ser datetime e frequência diária
serie = consumo_diario.set_index('date')['consumption_kwh']
serie = serie.asfreq('D')

# Decomposição (ajuste o período conforme necessário, ex: 7 para semanal)
resultado = seasonal_decompose(serie, model='additive', period=7)

# Gráfico da tendência (amarelo)
fig_trend = go.Figure()
fig_trend.add_trace(go.Scatter(
    x=serie.index, 
    y=resultado.trend, 
    mode='lines', 
    name='Tendência',
    line=dict(color='orange')
))
fig_trend.update_layout(title='Tendência do Consumo Diário', xaxis_title='Data', yaxis_title='Tendência (kWh)', template='simple_white')
fig_trend.show()

# Gráfico da sazonalidade (amarelo)
fig_seasonal = go.Figure()
fig_seasonal.add_trace(go.Scatter(
    x=serie.index, 
    y=resultado.seasonal, 
    mode='lines', 
    name='Sazonalidade',
    line=dict(color='orange')
))
fig_seasonal.update_layout(title='Sazonalidade do Consumo Diário', xaxis_title='Data', yaxis_title='Sazonalidade (kWh)', template='simple_white')
fig_seasonal.show()

# Gráfico do resíduo (amarelo)
fig_resid = go.Figure()
fig_resid.add_trace(go.Scatter(
    x=serie.index, 
    y=resultado.resid, 
    mode='lines', 
    name='Resíduo',
    line=dict(color='orange')
))
fig_resid.update_layout(title='Resíduo do Consumo Diário', xaxis_title='Data', yaxis_title='Resíduo (kWh)', template='simple_white')
fig_resid.show()

In [32]:
# Está decomposição mostra a tendência, sazonalidade e resíduo do consumo diário de energia.
# O gráfico de tendência reafirma a informação extraída do gráfico consumo total diário de energia de que não há alta ou queda consistente ao longo dos meses.
# O consumo é sazonal semanalmente.

# Clientes  x Clima X Consumo

In [33]:
df = (
    consumo
    .merge(
        clientes, 
        on='client_id', 
        how='inner'
    )
    .merge(
        clima, 
        on=['region', 'date'], 
        how='inner'
    )
)

display(df.head(10))

,client_id,date,consumption_kwh,region,temperature,humidity
0,C0000,2023-01-01,18.64,Norte,27.090625,59.068750
1,C0000,2023-01-02,16.63,Norte,27.090625,59.206250
2,C0000,2023-01-03,18.11,Norte,24.325000,59.365625
3,C0000,2023-01-04,18.25,Norte,23.987500,57.340625
4,C0000,2023-01-05,19.81,Norte,25.200000,59.921875
5,C0000,2023-01-06,15.87,Norte,24.896875,60.596875
6,C0000,2023-01-07,20.30,Norte,22.606250,57.281250
7,C0000,2023-01-08,19.35,Norte,24.739062,60.884375
8,C0000,2023-01-09,18.30,Norte,26.871875,61.200000
9,C0000,2023-01-10,13.34,Norte,24.846875,61.028125


In [34]:
# Gráfico de correlação das variáveis numéricas do df por região

for reg in df['region'].unique():
    df_reg = df[df['region'] == reg]
    df_corr = df_reg.select_dtypes(include=[np.number]).corr()

    fig = ff.create_annotated_heatmap(
        z=df_corr.values,
        x=list(df_corr.columns),
        y=list(df_corr.index),
        colorscale='Viridis',
        showscale=True,
        reversescale=True,
        annotation_text=df_corr.round(2).values
    )

    fig.update_layout(
        title=f'Mapa de Calor da Correlação das Variáveis Numéricas - Região {reg}',
        template='simple_white'
    )
    fig.show()

In [35]:
bins = [0, 22, 24, 26, 28, 30, np.inf]
labels = ['<=22°C', '22-24°C', '24-26°C', '26-28°C', '28-30°C', '>30°C']
df['faixa_temp'] = pd.cut(df['temperature'], bins=bins, labels=labels, include_lowest=True)

fig = px.box(
    df,
    x='faixa_temp',
    y='consumption_kwh',
    color='faixa_temp',
    category_orders={'faixa_temp': labels},
    title='Boxplot do Consumo por Faixa de Temperatura',
    labels={'faixa_temp': 'Faixa de Temperatura', 'consumption_kwh': 'Consumo (kWh)'},
    template='simple_white'
)
fig.update_layout(showlegend=False)
fig.show()

In [36]:
# Cria faixas de umidade
bins_hum = [0, 50, 55, 60, 65, 70, np.inf]
labels_hum = ['<=50%', '50-55%', '55-60%', '60-65%', '65-70%', '>70%']
df['faixa_hum'] = pd.cut(df['humidity'], bins=bins_hum, labels=labels_hum, include_lowest=True)

fig = px.box(
    df,
    x='faixa_hum',
    y='consumption_kwh',
    color='faixa_hum',
    category_orders={'faixa_hum': labels_hum},
    title='Boxplot do Consumo por Faixa de Umidade',
    labels={'faixa_hum': 'Faixa de Umidade', 'consumption_kwh': 'Consumo (kWh)'},
    template='simple_white'
)
fig.update_layout(showlegend=False)
fig.show()

In [37]:
# Não há correlações fortes entre consumo de energia, temperatura e umidade.

In [38]:
for reg in df['region'].unique():
    # Agrupa por dia e soma o consumo da região
    df_reg = (
        df[df['region'] == reg]
        .groupby('date', as_index=False)['consumption_kwh']
        .sum()
        .sort_values('date')
    )
    x = df_reg['date'].map(pd.Timestamp.toordinal)
    y = df_reg['consumption_kwh']
    smoothed = lowess(y, x, frac=0.05)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_reg['date'],
        y=y,
        mode='lines',
        name='Original',
        line=dict(color='black', width=1),
        opacity=0.4
    ))
    fig.add_trace(go.Scatter(
        x=df_reg['date'],
        y=smoothed[:, 1],
        mode='lines',
        name='LOWESS',
        line=dict(color='red', width=2)
    ))
    fig.update_layout(
        title=f'Tendência de Consumo Diário (Soma) com LOWESS - Região {reg}',
        xaxis_title='Data',
        yaxis_title='Consumo Total (kWh)',
        template='simple_white'
    )
    fig.show()

# LOWESS é um suavizador não-paramétrico, que ajusta uma curva suave aos dados sem assumir um formato fixo

In [39]:
# O consumo não é estático: há variações sazonais claras que um modelo preditivo deve considerar.
# Fatores como temperatura e umidade como variáveis explicativas na modelagem. Mesmo não apresentando correlações fortes, esses fatores influenciam o consumo de energia.


In [40]:
# Consumo total por região
consumo_por_regiao = (
    df.groupby('region')['consumption_kwh']
    .sum()
    .sort_values(ascending=False)
)

print(consumo_por_regiao)

# Gráfico para visualização
fig = px.bar(
    consumo_por_regiao.reset_index(),
    x='region',
    y='consumption_kwh',
    title='Consumo Total de Energia por Região',
    labels={'region': 'Região', 'consumption_kwh': 'Consumo Total (kWh)'},
    text='consumption_kwh',
    template='simple_white'
)
fig.show()

region
Norte     79512.86
Leste     48505.37
Centro    43051.46
Oeste     42729.54
Sul       39666.62
Name: consumption_kwh, dtype: float64


In [41]:
# região Norte se destaca com o maior consumo total de energia, seguido pelas regiões Leste e Centro.
# Norte se destaca por ter o maior número de clientes, o que pode explicar seu consumo elevado.

In [42]:
# Calcula o consumo médio por região
consumo_medio_regiao = (
    df.groupby('region')['consumption_kwh']
    .mean()
    .sort_values(ascending=False)
)

print(consumo_medio_regiao)

# Gráfico para visualização
fig = px.bar(
    consumo_medio_regiao.reset_index(),
    x='region',
    y='consumption_kwh',
    title='Consumo Médio de Energia por Região',
    labels={'region': 'Região', 'consumption_kwh': 'Consumo Médio (kWh)'},
    text='consumption_kwh',
    template='simple_white'
)
fig.show()

region
Oeste     15.825756
Leste     14.970793
Centro    14.948424
Sul       14.691341
Norte     14.249616
Name: consumption_kwh, dtype: float64


In [43]:
# Todas as regiões apresentam consumo médio de energia semelhante, com a região Norte ligeiramente à frente.